In [2]:
#dependencies 
from api_key import Auth_Token
import requests
import json
from pandas.io.json import json_normalize
from flatten_json import flatten
import pandas as pd
import regex 

In [31]:
#Options for Params
    #"q"
search = 'use the input function ->' #input() if creating a dasboard. 
    # "year"
model_year = 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023
    # "category"
cat = "road"
    # "subcategory"
sub_cat = "gravel", "cyclocross", "utility","general-urban","touring","commuter"
    # "makerId"
Brand = "salsa", "surly", "trek", "specialized", "giant", "cannondale", "kona", "rockymountain", "santacruz", "norco", "yeti", "niner", "cannondale", "scott", "orbea", "pivot", "cube", "bmc"
    # "isEbike"
eBike = "false"
    #"isFrameset"
frameset = "false"
    #"include"
include = "*"

GEO = []

params = {
    "limit" : 200,
    "cursor" : 'start',
    # "makerId" : Brand,
    "category" : cat,
    # "subcategory" : sub_cat,  
    "include" : include,
    # "isFrameset" : frameset,

}

while True: 
    BASE_URL = 'https://api.99spokes.com/v1/bikes'
    headers = {f'Authorization': "Bearer {}".format(Auth_Token)}
    response = requests.get(BASE_URL, headers=headers, params=params)
    bikes = response.json()
    if bikes['nextCursor'] == None:
        print("Complete")
        break
    GEO.extend(bikes['items'])
    nextCursor = bikes['nextCursor']
    params = {
    "limit" : 200,
    "cursor" : nextCursor,
    "makerId" : Brand,
    "category" : cat, 
    "include" : include,
}


Complete


In [32]:
#flatten all the data and place every list into columns. 
data = GEO
dict_flattened = (flatten(record, '.') for record in data)
df = pd.DataFrame(dict_flattened)
print(f"Data Returned this many bikes:", len(df))

Data Returned this many bikes: 7200


In [33]:
#clean up the data to better analyze (Example:  Removing the inch callout in tire size)

df["components.tires.width"] = df["components.tires.width"].str.replace('"',"", regex=True).str.replace('c',"", regex=True)

df.to_csv("check.csv")

In [34]:
#reduce down all the unwanted columns and rename them. 
reduced_df = df[['maker', 'year', 'model', 'family', 'category', 'subcategory',
                'sizes.2.geometry.source.stackMM',
                'sizes.2.geometry.source.reachMM',
                'sizes.2.geometry.computed.stackReachRatio',
                'sizes.2.geometry.source.topTubeLengthMM',
                'sizes.2.geometry.source.seatTubeLengthMM',
                'sizes.2.geometry.source.seatTubeAngle', 
                'sizes.2.geometry.source.headTubeLengthMM', 
                'sizes.2.geometry.source.headTubeAngle', 
                'sizes.2.geometry.source.chainstayLengthMM', 
                'sizes.2.geometry.source.bottomBracketDropMM', 
                'sizes.2.geometry.source.bottomBracketHeightMM',
                'sizes.2.geometry.source.frontCenterMM', 
                'sizes.2.geometry.source.wheelbaseMM', 
                'sizes.2.geometry.source.standoverHeightMM',
                'sizes.2.geometry.source.rakeMM',
                'sizes.2.geometry.source.trailMM',
                ]]

geo_df = pd.DataFrame(reduced_df.rename(columns={'maker':'Brand', 'year':'Model_Year', 'model':'Model', 'family':'Platform', 'category':'Category', 'subcategory':'Sub_Cat', 
                'sizes.2.geometry.source.stackMM':'Stack',
                'sizes.2.geometry.source.reachMM':'Reach',
                'sizes.2.geometry.computed.stackReachRatio':'SR_ratio',
                'sizes.2.geometry.source.topTubeLengthMM':'TT_length',
                'sizes.2.geometry.source.seatTubeLengthMM':'ST_length',
                'sizes.2.geometry.source.headTubeLengthMM':'HT_length', 
                'sizes.2.geometry.source.seatTubeAngle':'ST_Angle', 
                'sizes.2.geometry.source.headTubeAngle':'HT_Angle', 
                'sizes.2.geometry.source.chainstayLengthMM':'CS_length', 
                'sizes.2.geometry.source.bottomBracketDropMM':'BB_drop',
                'sizes.2.geometry.source.bottomBracketHeightMM':'BB_height',
                'sizes.2.geometry.source.wheelbaseMM': 'Wheelbase',
                'sizes.2.geometry.source.standoverHeightMM':'Standover',
                'sizes.2.geometry.source.rakeMM':'Rake', 
                'sizes.2.geometry.source.trailMM':'Trail',

        
                }))     
geo_df                   

,Brand,Model_Year,Model,Platform,Category,Sub_Cat,Stack,Reach,SR_ratio,TT_length,...,HT_length,HT_Angle,CS_length,BB_drop,BB_height,sizes.2.geometry.source.frontCenterMM,Wheelbase,Standover,Rake,Trail
0,Trek,2023,Checkpoint ALR 5 Driftless,Checkpoint,road,gravel,571.00,403.00,1.417,570.0,...,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
1,Trek,2023,Checkpoint SL 6 AXS,Checkpoint,road,gravel,571.00,403.00,1.417,570.0,...,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
2,Trek,2023,Checkpoint SL 7 AXS,Checkpoint,road,gravel,571.00,403.00,1.417,570.0,...,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
3,Trek,2023,Checkpoint SLR 6 AXS,Checkpoint,road,gravel,571.00,403.00,1.417,570.0,...,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
4,Trek,2023,Checkpoint SLR 7 AXS,Checkpoint,road,gravel,571.00,403.00,1.417,570.0,...,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,Kona,2017,Esatto Ti Disc,Esatto,road,endurance,575.00,380.00,1.513,550.0,...,165.0,72.0,422.0,74.0,NaN,593.0,1005.0,772.00,45.0,NaN
7196,Kona,2016,Esatto Ti Disc,Esatto,road,endurance,575.00,380.00,1.513,550.0,...,165.0,72.0,422.0,74.0,NaN,593.0,1005.0,772.00,45.0,NaN
7197,Kona,2015,ESATTO TI,ESATTO,road,endurance,577.00,379.00,1.522,550.0,...,165.0,72.0,418.0,74.0,NaN,593.0,1000.0,785.00,44.0,NaN
7198,Salsa,2021,Stormchaser Frameset,Stormchaser,road,gravel,562.61,374.06,1.504,545.0,...,125.0,70.0,435.0,70.0,NaN,NaN,1037.0,756.48,51.0,NaN


In [66]:
#reduce down all the unwanted columns and rename them. 
reduced_df = df[['year','family','subcategory',
                'sizes.2.geometry.source.stackMM',
                'sizes.2.geometry.source.reachMM',
                'sizes.2.geometry.computed.stackReachRatio',
                'sizes.2.geometry.source.topTubeLengthMM',
                'sizes.2.geometry.source.seatTubeLengthMM',
                'sizes.2.geometry.source.seatTubeAngle', 
                'sizes.2.geometry.source.headTubeLengthMM', 
                'sizes.2.geometry.source.headTubeAngle', 
                'sizes.2.geometry.source.chainstayLengthMM', 
                'sizes.2.geometry.source.bottomBracketDropMM', 
                'sizes.2.geometry.source.bottomBracketHeightMM',
                'sizes.2.geometry.source.frontCenterMM', 
                'sizes.2.geometry.source.wheelbaseMM', 
                'sizes.2.geometry.source.standoverHeightMM',
                'sizes.2.geometry.source.rakeMM',
                'sizes.2.geometry.source.trailMM',
                ]]

geoML_df = pd.DataFrame(reduced_df.rename(columns={'year':'model_year','family':'Platform', 'subcategory':'Sub_Cat',
                'sizes.2.geometry.source.stackMM':'Stack',
                'sizes.2.geometry.source.reachMM':'Reach',
                'sizes.2.geometry.computed.stackReachRatio':'SR_ratio',
                'sizes.2.geometry.source.topTubeLengthMM':'TT_length',
                'sizes.2.geometry.source.seatTubeLengthMM':'ST_length',
                'sizes.2.geometry.source.headTubeLengthMM':'HT_length', 
                'sizes.2.geometry.source.seatTubeAngle':'ST_Angle', 
                'sizes.2.geometry.source.headTubeAngle':'HT_Angle', 
                'sizes.2.geometry.source.chainstayLengthMM':'CS_length', 
                'sizes.2.geometry.source.bottomBracketDropMM':'BB_drop',
                'sizes.2.geometry.source.bottomBracketHeightMM':'BB_height',
                'sizes.2.geometry.source.wheelbaseMM': 'Wheelbase',
                'sizes.2.geometry.source.standoverHeightMM':'Standover',
                'sizes.2.geometry.source.rakeMM':'Rake', 
                'sizes.2.geometry.source.trailMM':'Trail',

        
                }))     
geoML_df 

,model_year,Platform,Sub_Cat,Stack,Reach,SR_ratio,TT_length,ST_length,ST_Angle,HT_length,HT_Angle,CS_length,BB_drop,BB_height,sizes.2.geometry.source.frontCenterMM,Wheelbase,Standover,Rake,Trail
0,2023,Checkpoint,gravel,571.00,403.00,1.417,570.0,540.0,73.2,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
1,2023,Checkpoint,gravel,571.00,403.00,1.417,570.0,540.0,73.2,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
2,2023,Checkpoint,gravel,571.00,403.00,1.417,570.0,540.0,73.2,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
3,2023,Checkpoint,gravel,571.00,403.00,1.417,570.0,540.0,73.2,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
4,2023,Checkpoint,gravel,571.00,403.00,1.417,570.0,540.0,73.2,125.0,71.8,435.0,76.0,NaN,NaN,1041.0,789.00,NaN,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,2017,Esatto,endurance,575.00,380.00,1.513,550.0,540.0,73.5,165.0,72.0,422.0,74.0,NaN,593.0,1005.0,772.00,45.0,NaN
7196,2016,Esatto,endurance,575.00,380.00,1.513,550.0,540.0,73.5,165.0,72.0,422.0,74.0,NaN,593.0,1005.0,772.00,45.0,NaN
7197,2015,ESATTO,endurance,577.00,379.00,1.522,550.0,540.0,73.5,165.0,72.0,418.0,74.0,NaN,593.0,1000.0,785.00,44.0,NaN
7198,2021,Stormchaser,gravel,562.61,374.06,1.504,545.0,475.0,73.0,125.0,70.0,435.0,70.0,NaN,NaN,1037.0,756.48,51.0,NaN


In [64]:
geoML_df .drop_duplicates('Platform', inplace=True),
geoML_df .dropna(subset=['Stack', 'TT_length', 'ST_length','ST_Angle','HT_length','HT_Angle','BB_drop'], inplace=True)
geoML_df  = geoML_df [~geoML_df['Sub_Cat'].str.contains('triathlon')]
geoML_df.to_csv("GEOmL.csv") 

In [52]:
geo_df.drop_duplicates('Platform', inplace=True),
geo_df.dropna(subset=['Stack', 'TT_length', 'ST_length','ST_Angle','HT_length','HT_Angle','BB_drop'], inplace=True)
geo_df = geo_df[~geo_df['Sub_Cat'].str.contains('triathlon')]
geo_df

,Brand,Model_Year,Model,Platform,Category,Sub_Cat,Stack,Reach,SR_ratio,TT_length,...,HT_length,HT_Angle,CS_length,BB_drop,BB_height,sizes.2.geometry.source.frontCenterMM,Wheelbase,Standover,Rake,Trail
0,Trek,2023,Checkpoint ALR 5 Driftless,Checkpoint,road,gravel,571.0,403.0,1.417,570.0,...,125.0,71.80,435.0,76.0,NaN,NaN,1041.0,789.0,NaN,70.0
14,Specialized,2023,Diverge Comp E5,Diverge,road,gravel,592.0,383.0,1.546,558.0,...,116.0,71.25,432.0,80.0,270.0,620.0,NaN,738.0,55.0,61.0
15,Canyon,2023,Grail CF SL 7 Rapha,Grail,road,gravel,513.0,405.0,1.267,547.0,...,78.0,71.00,425.0,75.0,NaN,NaN,1020.0,805.0,NaN,NaN
16,Canyon,2023,Endurace CF SL 8,Endurace,road,endurance,532.0,370.0,1.438,527.0,...,115.0,71.00,415.0,75.0,NaN,NaN,982.0,750.0,NaN,NaN
33,Trek,2023,Émonda ALR 5,Émonda,road,race,541.0,386.0,1.402,543.0,...,131.0,73.00,410.0,70.0,NaN,NaN,981.0,744.0,NaN,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6421,Norco,2022,Section S Frame/Fork,Section S Frame/Fork,road,gravel,555.0,386.0,1.438,548.0,...,125.0,72.25,415.0,72.5,271.5,NaN,1002.0,717.0,51.0,56.0
6593,Specialized,2013,S-Works Tarmac SL4 OSBB Frameset,Tarmac SL4 OSBB,road,race,543.0,387.0,1.403,548.0,...,140.0,73.00,405.0,69.0,NaN,583.0,978.0,775.0,45.0,57.0
6962,Specialized,2013,S-Works Venge OSBB Frameset,Venge OSBB,road,aero,544.0,386.0,1.409,548.0,...,140.0,73.00,405.0,69.0,NaN,583.0,978.0,789.0,45.0,57.0
7093,Specialized,2017,AWOL Expert Frameset,AWOL Expert,road,touring,617.0,392.0,1.574,575.0,...,185.0,72.00,455.0,70.0,295.5,626.0,1072.0,794.0,50.0,66.0


In [53]:
na_count = geo_df.isna().sum()

print(na_count)

Brand                                      0
Model_Year                                 0
Model                                      0
Platform                                   0
Category                                   0
Sub_Cat                                    0
Stack                                      0
Reach                                      0
SR_ratio                                   0
TT_length                                  0
ST_length                                  0
ST_Angle                                   0
HT_length                                  0
HT_Angle                                   0
CS_length                                  0
BB_drop                                    0
BB_height                                128
sizes.2.geometry.source.frontCenterMM    122
Wheelbase                                  2
Standover                                 18
Rake                                      66
Trail                                    107
dtype: int

In [55]:
geo_df.to_csv("GEO.csv")  